# Setup

In [ ]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get update  &> /dev/null


apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip  &> /dev/null

# Boost libraries
apt-get install libboost-all-dev  &> /dev/null

In [ ]:
!pip install vizdoom --quiet
!pip install ray --quiet
!pip install ray['rllib'] --quiet
!pip install Ipython --upgrade --quiet

In [ ]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project'
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(system_path)

In [ ]:
from src.vizdoom_gym.envs.VizDoomEnv import VizdoomEnv
from src.vizdoom_gym.envs.VizDoomEnv_def import VizDoomVeryDenseReward

In [ ]:
from ray.tune.registry import register_env
import gym
import os
import ray
import ray.rllib.agents.ppo as ppo
import shutil
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device, "\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# Initialize Ray

In [ ]:
#need this to load vizdoom module 
system_path = '/content/drive/MyDrive/GitHub/INM363-Project/src' 
sys.path.append(system_path)

#need this to use gpu on ray 
os.environ['PYTHONPATH'] = '/content/drive/MyDrive/GitHub/INM363-Project' 
os.environ['PYTHONPATH']

In [ ]:
#chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/icm/dense_new_pattern"
#shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)

ray.shutdown()
print("Shutdown ray")

# start Ray -- add `local_mode=True` here for debugging
ray.init(ignore_reinit_error=True,  num_cpus =2, num_gpus = 1) #local_mode=True,

#ray.init(num_cpus= 2, num_gpus=1)

print("Initialized ray")

# register the custom environment
select_env = "VizDoomVeryDenseReward-v0"

register_env(select_env, lambda config: VizDoomVeryDenseReward())
#register_env(select_env, lambda config: VizdoomEnv())

print("registered environment")


Training config

In [ ]:
# configure the environment and create agent
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config["framework"] = "torch"
config["model"] = {"dim": 42, 
                   "grayscale": True,
                   }
config["num_gpus"] = 1
config["preprocessor_pref"] = "rllib"
config['explore'] = True 


In [ ]:

#activating curiosity as the exploration class : https://docs.ray.io/en/latest/rllib/rllib-algorithms.html

#set to 0 because of: https://discuss.ray.io/t/scaling-curiosity-like-exploration-modules-on-multiple-workers/2267
config["num_workers"] = 0 

config["exploration_config"] = {
    "type": "Curiosity",  # <- Use the Curiosity module for exploring.
    "eta": 0.0001, #0.001,  # Weight for intrinsic rewards before being added to extrinsic ones.
    "lr": 0.001,  # Learning rate of the curiosity (ICM) module.
    "feature_dim": 288,  # Dimensionality of the generated feature vectors.
    # Setup of the feature net (used to encode observations into feature (latent) vectors).
    "feature_net_config": {
        "fcnet_hiddens": [],
        "fcnet_activation": "relu",
    },
    "inverse_net_hiddens": [256],  # Hidden layers of the "inverse" model.
    "inverse_net_activation": "relu",  # Activation of the "inverse" model.
    "forward_net_hiddens": [256],  # Hidden layers of the "forward" model.
    "forward_net_activation": "relu",  # Activation of the "forward" model.
    "beta": 0.2,  # Weight for the "forward" loss (beta) over the "inverse" loss (1.0 - beta).
    "sub_exploration": {
        "type": "StochasticSampling",
    }
}


"""
used vf_clip = 400 for easy and new_dense settings 
= 600 for sparse settings 
= 10 (deafult) for easy no reward and no reward settings 
"""

config["vf_clip_param"] = 400



agent = ppo.PPOTrainer(config, env=select_env)

print("created agent")

**Training** loop

In [ ]:
import pandas as pd
import time 

cols = ["checkpoint", "eps_reward_min", "eps_reward_mean", "eps_reward_max", "eps_len_mean", "episodes_this_iter"]
results_df = pd.DataFrame(columns = cols) 

In [ ]:
#restore checkpoint if needed 
chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/icm/dense_new_pattern"
chkpt_file  = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/icm/dense_new_pattern/checkpoint_001100"
agent.restore(chkpt_file)

In [ ]:
#@title
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f}"
start_n = 300
n_iter = 600

print("started training loop")
time_start = time.time() 

chkpt_root = chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/icm/dense_new_pattern"


# train a policy with RLlib using PPO
for n in range(start_n, n_iter):
  
    result = agent.train()

    #change this to  10 or 20 
    if (n+1) % 30 == 0 or n == 0: 
      chkpt_file = agent.save(chkpt_root)
      print(f"Saved checkpoint {n+1} at {chkpt_file}")

    print(status.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"]
    ))

    #save metrics
    row = {'checkpoint': n+1,
       "eps_reward_min": result["episode_reward_min"],
       "eps_reward_mean": result["episode_reward_mean"],
       "eps_reward_max": result["episode_reward_max"],
       "eps_len_mean": result["episode_len_mean"],
       "episodes_this_iter": result["episodes_this_iter"]
       }
    results_df = results_df.append(row, ignore_index = True)


print(f"Total time elapsed: {(time.time()-time_start)/60}")

print("ending training loop")

ray.shutdown()
print("shutdown ray")

# save results file

In [ ]:
from pathlib import Path 

fname = chkpt_root + '/result_2.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
results_df.to_csv(fpath)
print(f"Saved results file to {fname}")


In [ ]:
import pandas as pandas
import os 
from pathlib import Path 


chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/icm/dense_new_pattern" 

results_pth = chkpt_root + '/' + 'result_1.csv'
df1 = pd.read_csv(results_pth) 
df1.tail()


results_pth = chkpt_root + '/' + 'result_2.csv'
df2 = pd.read_csv(results_pth) 
df2.tail()

In [ ]:

frames = [df1, df2]
df = pd.concat(frames, ignore_index = True)
df.tail()


fname = chkpt_root + '/result.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
df.to_csv(fpath)
print(f"Saved results file to {fname}")